In [1]:
!pip install scikeras

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 76.1 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 0.22.0 requires google-cloud-bigquery[bqstorage,pandas]>=3.10.0, but you have google-cloud-bigquery 2.34.4 which is incompatible.
bigframes 0.22.0 requires google-cloud-storage>=2.0.0, but you have google-cloud-storage 1.44.0 which is incompatible.
bigframes 0.22.0 requires pandas<2.1.4,>=1.5.0, but you have pandas 2.2.2 which is incompatible.
dataproc-jupyter-plugin 0.1.79 requires pydantic~=1.10.0, but you have pydantic 2.8.2 which is incompatible.
spaghetti 1.7.6 requires shapely>=2.0.1, but you have shapely 1.8.5.post1 which is incompatible.
spopt 0.

In [2]:
import pandas as pd
import time
import tensorflow.keras.backend as k
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, InputLayer
from scikeras.wrappers import KerasRegressor 
from sklearn.model_selection import cross_val_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

In [3]:
ds = pd.read_csv('/kaggle/input/autos-card12/autos.csv', encoding = 'ISO-8859-1')

In [4]:
ds

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-24 11:52:17,Golf_3_1.6,privat,Angebot,480,test,NaN,1993,manuell,0,golf,150000,0,benzin,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,A5_Sportback_2.7_Tdi,privat,Angebot,18300,test,coupe,2011,manuell,190,NaN,125000,5,diesel,audi,ja,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,"Jeep_Grand_Cherokee_""Overland""",privat,Angebot,9800,test,suv,2004,automatik,163,grand,125000,8,diesel,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,GOLF_4_1_4__3TÜRER,privat,Angebot,1500,test,kleinwagen,2001,manuell,75,golf,150000,6,benzin,volkswagen,nein,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,Skoda_Fabia_1.4_TDI_PD_Classic,privat,Angebot,3600,test,kleinwagen,2008,manuell,69,fabia,90000,7,diesel,skoda,nein,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
371523,2016-03-14 17:48:27,Suche_t4___vito_ab_6_sitze,privat,Angebot,2200,test,NaN,2005,NaN,0,NaN,20000,1,NaN,sonstige_autos,NaN,2016-03-14 00:00:00,0,39576,2016-04-06 00:46:52
371524,2016-03-05 19:56:21,Smart_smart_leistungssteigerung_100ps,privat,Angebot,1199,test,cabrio,2000,automatik,101,fortwo,125000,3,benzin,smart,nein,2016-03-05 00:00:00,0,26135,2016-03-11 18:17:12
371525,2016-03-19 18:57:12,Volkswagen_Multivan_T4_TDI_7DC_UY2,privat,Angebot,9200,test,bus,1996,manuell,102,transporter,150000,3,diesel,volkswagen,nein,2016-03-19 00:00:00,0,87439,2016-04-07 07:15:26
371526,2016-03-20 19:41:08,VW_Golf_Kombi_1_9l_TDI,privat,Angebot,3400,test,kombi,2002,manuell,100,golf,150000,6,diesel,volkswagen,NaN,2016-03-20 00:00:00,0,40764,2016-03-24 12:45:21


# Removendo colunas não pertinentes e não convenientes

In [5]:
ds.columns

Index(['dateCrawled', 'name', 'seller', 'offerType', 'price', 'abtest',
       'vehicleType', 'yearOfRegistration', 'gearbox', 'powerPS', 'model',
       'kilometer', 'monthOfRegistration', 'fuelType', 'brand',
       'notRepairedDamage', 'dateCreated', 'nrOfPictures', 'postalCode',
       'lastSeen'],
      dtype='object')

In [6]:
toDrop = ['dateCrawled', 'name', 'seller', 'offerType', 'dateCreated', 'nrOfPictures', 'postalCode', 'lastSeen']
for i in toDrop:
    ds = ds.drop(i, axis=1)

In [7]:
ds

,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage
0,480,test,NaN,1993,manuell,0,golf,150000,0,benzin,volkswagen,NaN
1,18300,test,coupe,2011,manuell,190,NaN,125000,5,diesel,audi,ja
2,9800,test,suv,2004,automatik,163,grand,125000,8,diesel,jeep,NaN
3,1500,test,kleinwagen,2001,manuell,75,golf,150000,6,benzin,volkswagen,nein
4,3600,test,kleinwagen,2008,manuell,69,fabia,90000,7,diesel,skoda,nein
...,...,...,...,...,...,...,...,...,...,...,...,...
371523,2200,test,NaN,2005,NaN,0,NaN,20000,1,NaN,sonstige_autos,NaN
371524,1199,test,cabrio,2000,automatik,101,fortwo,125000,3,benzin,smart,nein
371525,9200,test,bus,1996,manuell,102,transporter,150000,3,diesel,volkswagen,nein
371526,3400,test,kombi,2002,manuell,100,golf,150000,6,diesel,volkswagen,NaN


# Lidando com valores inconsistentes ou nulos

Retirando registros com preços inconsistentes:

In [8]:
ds = ds.loc[ds['price'] > 10]
ds = ds.loc[ds['price'] < 350000]

Substituindo valores nulos em atributos categóricos pela moda:

In [9]:
ds.isnull().sum()

price                      0
abtest                     0
vehicleType            33546
yearOfRegistration         0
gearbox                17236
powerPS                    0
model                  17967
kilometer                  0
monthOfRegistration        0
fuelType               29391
brand                      0
notRepairedDamage      65986
dtype: int64

In [10]:
values = {
    'vehicleType': ds['vehicleType'].mode()[0],
    'gearbox': ds['gearbox'].mode()[0],
    'model': ds['model'].mode()[0],
    'fuelType': ds['fuelType'].mode()[0],
    'notRepairedDamage': ds['notRepairedDamage'].mode()[0]
}

In [11]:
ds = ds.fillna(value = values)

In [12]:
ds.isnull().sum()

price                  0
abtest                 0
vehicleType            0
yearOfRegistration     0
gearbox                0
powerPS                0
model                  0
kilometer              0
monthOfRegistration    0
fuelType               0
brand                  0
notRepairedDamage      0
dtype: int64

# Criando os conjuntos previsores e de classe

In [13]:
X = ds.iloc[:, 1:12].values 
y = ds.iloc[:, 0].values

In [14]:
ds.columns

Index(['price', 'abtest', 'vehicleType', 'yearOfRegistration', 'gearbox',
       'powerPS', 'model', 'kilometer', 'monthOfRegistration', 'fuelType',
       'brand', 'notRepairedDamage'],
      dtype='object')

Agora, uma instância de ColumnTransformer será criada para aplicar a função OneHotEncoder nas colunas especificadas de um DataFrame.

In [15]:
onehotencoder = ColumnTransformer(transformers = [("OneHot", OneHotEncoder(), [0, 1, 3, 5, 8, 9, 10])], remainder = 'passthrough')

In [16]:
X = onehotencoder.fit_transform(X) #aplica a transformação à X

In [17]:
X.shape

(359291, 316)

# Criando a estrutura da rede neural

In [18]:
def create_network():
    k.clear_session()
    network = Sequential()
    network.add(InputLayer(shape = (316,)))
    network.add(Dense(units = 158, activation='relu'))
    network.add(Dense(units = 158, activation='relu'))
    network.add(Dense(units = 1, activation='linear'))

    network.compile(optimizer='adam', loss='mean_absolute_error', metrics=['mean_absolute_error'])
    return network

In [19]:
network = KerasRegressor(model = create_network, epochs = 100, batch_size=300)

In [20]:
results = cross_val_score(estimator = network, X = X, y = y, 
                         cv = 5, scoring = 'neg_mean_absolute_error')

Epoch 1/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 7s 5ms/step - loss: 4246.5718 - mean_absolute_error: 4246.5718
Epoch 2/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - loss: 3481.8909 - mean_absolute_error: 3481.8909
Epoch 3/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - loss: 3262.8872 - mean_absolute_error: 3262.8872
Epoch 4/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - loss: 3034.8162 - mean_absolute_error: 3034.8162
Epoch 5/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - loss: 2933.0029 - mean_absolute_error: 2933.0027
Epoch 6/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - loss: 2845.6597 - mean_absolute_error: 2845.6597
Epoch 7/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - loss: 2790.0645 - mean_absolute_error: 2790.0645
Epoch 8/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - loss: 2748.5239 - mean_absolute_error: 2748.5239
Epoch 9/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - loss: 2731.3254 - mean_absolute_error: 2731.3254
Epoch 10/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - loss: 26

In [21]:
results

array([-2197.36617453, -2237.22226802, -2450.78351189, -2484.97495774,
       -2370.22990334])

In [22]:
abs(results.mean())

2348.1153631041175

In [23]:
results.std()

113.82701498801474